# Facebook

## Get the data

In [ ]:
page_url = 'https://graph.facebook.com/v2.8/Google/feed?fields=id,message,reactions,shares,from,caption,created_time,likes.summary(true)' 
comments_url = 'https://graph.facebook.com/v2.8/{post_id}/comments?filter=stream&limit=100'

from pymongo import MongoClient

params = {'access_token': 'YOUR_ACCESS TOKEN'}
client = MongoClient('localhost:27017')
db = client.facebook
collection_posts = db.posts
collection_comments = db.comments

posts = requests.get(page_url, params = params)
posts = posts.json()

while True:
    try:
    ###Retrieve one post
        for element in posts['data']:
            collection_posts.insert(element)
            ####Retrieve all comments for this post
            this_comment_url = comments_url.replace("{post_id}",element['id'])
            comments = requests.get(this_comment_url, params = params).json()
            
            #loop through all comments until the response is empty (there are no more comments)
            while ('paging' in comments and 'cursors' in comments['paging'] and 'after' in comments['paging']['cursors']):
                ###Iterate through all comments
                for comment in comments['data']:
                    comment['post_id'] = element['id']
                    collection_comments.insert(comment)
                
                comments = requests.get(this_comment_url + '&after=' + comments['paging']['cursors']['after'], params = params).json()
 
        ####Go to the next page in feed
        posts = requests.get(posts['paging']['next']).json()
    except (KeyError, e):
        print(e)
        break

## Pull the data from MongoDB

In [20]:
import pandas as pd
import nltk

#create dataframes
df_posts = pd.DataFrame(posts_data)
df_posts.columns = ['message','created_time','likes','shares','post_id']
df_comments = pd.DataFrame(comments_data)
df_comments.columns = ['message','created_time','post_id']

## Pre-process the data

## Visualization

In [ ]:
from wordcloud import WordCloud
import datetime
import itertools
import matplotlib.pyplot as plt

def viz_wordcloud(dataframe,column_name):
 
    #Count words or phrases
 
    lst_tokens = list(itertools.chain.from_iterable(dataframe[column_name]))
    lst_phrases = [phrase.replace(" ","_") for phrase in lst_tokens]
    
    #Define your font_path according to the operating system paths
    wordcloud = WordCloud(font_path='/Library/Fonts/Verdana.ttf',background_color="white", max_words=2000, max_font_size=40, random_state=42).generate(" ".join(lst_phrases))
 
    # Display the generated image:
    # the matplotlib way:
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()


### Cleaning

In [63]:
CLEANING_LST = ['gulf','d','ban','persic']

#lst_phrases = [phrase for phrase in lst_phrases if not any(spam in phrase.lower() for spam in CLEANING_LST)]
#lst_phrases = [phrase.replace(" ","_") for phrase in lst_phrases if len(phrase) > 1 ]

def viz_wordcloud(dataframe,column_name):
 
    #Count words or phrases
    lst_tokens = list(itertools.chain.from_iterable(dataframe[column_name]))
    lst_phrases = [phrase.replace(" ","_") for phrase in lst_tokens if len(phrase) > 1 ]
    lst_phrases = [phrase for phrase in lst_phrases if not any(spam in phrase.lower() for spam in CLEANING_LST)]
    
   
    
    #Define your font_path according to the operating system paths
    wordcloud = WordCloud(font_path='/Library/Fonts/Verdana.ttf',background_color="white", max_words=2000, max_font_size=40, random_state=42).generate(" ".join(lst_phrases))
 
    # Display the generated image:
    # the matplotlib way:
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()



## Print results

In [22]:
# Function to print verbatims

def print_verbatims(df,nb_verbatim,keyword):
 
    verbatims = df[df['message'].str.contains(keyword)]
    
    for i,text in verbatims.head(nb_verbatim).iterrows():
        print(text['message'])

print_verbatims(df_comments,5,'google')

HurrayI luv <3 google
now this wat i'm talking about.. It's high time google got a page of their own in here !
Www.google.com
i love gothgoogle!! lol!!!! its a real website!!
I'm addicted to google. it makes me smarter.


## Time series analysis

## Wordcloud on time series

## Alchemy API